<a href="https://colab.research.google.com/github/samridhisharan/Crop-Prediction-Model/blob/main/Samridhi_Purnamita__Arizeet_Presha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce

In [ ]:
df=pd.read_csv('crop.csv')
df.sample(10)

,State,District,Crop,Year,Season,Area,Area Units,Production,Production Units,Yield
86740,Andhra Pradesh,WARANGAL,Sesamum,2008-09,Rabi,356.0,Hectare,89.0,Tonnes,0.250000
35092,Uttar Pradesh,ALLAHABAD,Groundnut,2001-02,Kharif,2.0,Hectare,2.0,Tonnes,1.000000
197502,Madhya Pradesh,SATNA,Barley,2013-14,Kharif,7552.0,Hectare,6891.0,Tonnes,0.912474
258627,Punjab,FARIDKOT,Barley,2017-18,Rabi,100.0,Hectare,300.0,Tonnes,3.000000
157932,Maharashtra,NAGPUR,Sunflower,2012-13,Rabi,900.0,Hectare,400.0,Tonnes,0.444444
13937,Jammu and Kashmir,ANANTNAG,Maize,2001-02,Kharif,23725.0,Hectare,23820.0,Tonnes,1.004004
17742,Kerala,ERNAKULAM,Arecanut,2003-04,Whole Year,5275.0,Hectare,3813.0,Tonnes,0.722844
269246,Uttar Pradesh,AMBEDKAR NAGAR,Bajra,2016-17,Kharif,8.0,Hectare,7.0,Tonnes,0.875000
215571,Tripura,KHOWAI,Sugarcane,2014-15,Whole Year,48.0,Hectare,2612.0,Tonnes,54.416667
91321,Bihar,MUNGER,Maize,2009-10,Summer,1583.0,Hectare,2951.0,Tonnes,1.864182


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 345407 entries, 0 to 345406
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   State             345407 non-null  object 
 1   District          345407 non-null  object 
 2   Crop              345375 non-null  object 
 3   Year              345407 non-null  object 
 4   Season            345406 non-null  object 
 5   Area              345374 non-null  float64
 6   Area Units        345407 non-null  object 
 7   Production        340414 non-null  float64
 8   Production Units  345407 non-null  object 
 9   Yield             345374 non-null  float64
dtypes: float64(3), object(7)
memory usage: 26.4+ MB


In [ ]:
for col in df.columns:  # Printing the no of columns along with the no of unique values presnt in them
  print(col, df[col].nunique())

State 36
District 729
Crop 56
Year 24
Season 6
Area 48006
Yield 197154
Production (Tonnes) 72881


In [ ]:
df['Production Units'].unique()

array(['Tonnes', 'Nuts', 'Bales'], dtype=object)

In [ ]:
df.sample(10)

,State,District,Crop,Year,Season,Area,Yield,Production (Tonnes),Year_Number
286519,2.220290,2.245669,1.956544,2019-01-01,Rabi,28.0,3.214286,90.0,2019
332704,2.116191,2.061444,1.172369,1999-01-01,Kharif,22.0,0.090909,2.0,1999
70481,2.762335,2.798165,1.465868,2005-01-01,Summer,14.0,0.571429,8.0,2005
16548,2.163787,2.065041,3.178040,2002-01-01,Year,2.0,13.000000,26.0,2002
334244,2.502564,2.567423,1.808260,1998-01-01,Kharif,7461.0,0.303981,2268.0,1998
271393,1.822313,1.878247,1.579802,2018-01-01,Kharif,24019.0,1.994005,47894.0,2018
148296,2.163787,2.213861,1.382844,2011-01-01,Kharif,196.0,2.025510,397.0,2011
216197,1.822313,1.689711,1.956544,2013-01-01,Rabi,1103.0,2.975521,3282.0,2013
18971,2.220290,2.211783,3.562835,2001-01-01,Year,124.0,0.338710,42.0,2001
328333,2.762335,2.814516,1.993197,1999-01-01,Rabi,320.0,0.218750,70.0,1999


In [ ]:
# If Year is in 'YYYY-YY' format, we'll convert it to 'YYYY'
df['Year'] = df['Year'].apply(lambda x: str(x).split('-')[0])

# Now, convert the 'Year' column to a datetime object
df['Year'] = pd.to_datetime(df['Year'], format='%Y')

# keep the year as a separate feature, extract it from the datetime
df['Year_Number'] = df['Year'].dt.year

# Display the updated DataFrame
print(df[['Year', 'Year_Number']].head())

        Year  Year_Number
0 2001-01-01         2001
1 2002-01-01         2002
2 2003-01-01         2003
3 2001-01-01         2001
4 2002-01-01         2002


In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Prepare data
X = df.drop(columns=['Season'])  # Drop the target column 'Season'
y = df['Season']

In [ ]:
# Feature Scaling
scaler = StandardScaler()
X[['Area', 'Production (Tonnes)', 'Yield', 'Year_Number']] = scaler.fit_transform(X[['Area', 'Production (Tonnes)', 'Yield', 'Year_Number']])

In [ ]:
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')

# Drop rows with NaN in 'Year' if any
df = df.dropna(subset=['Year'])
 model
#'Year' is an integer
df['Year'] = df['Year'].astype(int)

# Separate features and target
X = df.drop(columns=['Season'])
y = df['Season']

In [ ]:
# categorical columns
categorical_cols = ['State', 'District', 'Crop']
numeric_cols = ['Area', 'Production (Tonnes)', 'Yield', 'Year']  # Remove 'Year_Number' from here

#  transformers for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
        ('num', StandardScaler(), numeric_cols)
    ],
    remainder='passthrough'  # Keep any other columns as-is
)

In [1]:
#  pipeline that first preprocesses the data then fits the model
from sklearn.pipeline import Pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

NameError: name 'preprocessor' is not defined

In [ ]:
# Train the model
pipeline.fit(X_train, y_train)

# Predict and evaluate
y_pred = pipeline.predict(X_test)

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Cross-Validation
cv_scores = cross_val_score(pipeline, X, y, cv=5)
print("\nCross-Validation Accuracy: ", np.mean(cv_scores))

Accuracy: 0.9374077183636838

Classification Report:
               precision    recall  f1-score   support

      Autumn       0.79      0.80      0.80      1380
      Kharif       0.94      0.96      0.95     27712
        Rabi       0.94      0.92      0.93     20167
      Summer       0.82      0.82      0.82      4440
      Winter       0.84      0.81      0.83      1642
        Year       0.99      0.98      0.99     13741

    accuracy                           0.94     69082
   macro avg       0.89      0.88      0.89     69082
weighted avg       0.94      0.94      0.94     69082


Confusion Matrix:
 [[ 1107    15    19   104   130     5]
 [   11 26514   844   252     0    91]
 [   29  1132 18647   295    19    45]
 [  128   296   258  3641   104    13]
 [  123    11    36   121  1334    17]
 [    0   148    71     4     3 13515]]
